In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, rfft, rfftfreq
from scipy.io.wavfile import write, read 
from scipy.io import wavfile
import math
from time import sleep
from math import floor
import pyaudio
import wave
import random
import pandas as pd

In [250]:
#ЗАПИСЬ
filename = "recorded.wav"
chunk = 1024
FORMAT = pyaudio.paInt16
channels = 1
sample_rate = 44100
record_seconds = 10
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
frames = []
print("Recording")
for i in range(int(sample_rate / chunk * record_seconds)):
  data = stream.read(chunk)
  frames.append(data)
stream.stop_stream()
stream.close()
p.terminate()
wf = wave.open(filename, "wb")
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(sample_rate)
wf.writeframes(b"".join(frames))
wf.close()

ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave


Recording


In [246]:
class modulator:

    def __init__(self):
        self.sample_rate, self.freq0, self.freq1, self.change_time = 44100, 1000, 2000, 0.5

    def set_values(self, sample_rate, freq0, freq1, text, change_time):
        self.sample_rate, self.freq0, self.freq1, self.text, self.change_time = sample_rate, freq0, freq1, text, change_time
    
    def set_freq0(self, freq0):
       self.freq0 = freq0
    
    def set_freq1(self, freq1):
       self.freq1 = freq1
    
    def set_sample_rate(self, sample_rate):
       self.sample_rate = sample_rate
    
    def set_text(self, text):
       self.text = text
    
    def set_change_time(self, change_time):
       self.change_time = change_time

    @staticmethod
    def generate_sine_wave(freq, sample_rate, duration):
      x = np.linspace(0, duration, int(sample_rate*duration), endpoint=False)
      frequencies = x * freq
      y = np.sin((2 * np.pi) * frequencies)
      return x, y
    
    def signal_modulation(self): #Следует доработать, т.к неэффективно работает. Но это мало на что влияет, на этом этапе скорость не нужна
      change_time, text, freq0, freq1, sample_rate = self.change_time, self.text, self.freq0, self.freq1, self.sample_rate
      duration = change_time * len(text)
      num = int(duration / change_time)
      time = np.array([], dtype = np.int16)
      signal = np.array([], dtype = np.int16)
      for i in range(num):
        if text[i] == "0":
            freq = freq0
        if text[i] == "1":
            freq = freq1
        time_diff, data = self.generate_sine_wave(freq, sample_rate, change_time)
        signal = np.concatenate((signal, data), axis = None)
        time = np.concatenate((time, time_diff), axis = None)
      return time, signal
    
class wav_manager:
   
   def __init__(self):
    self.recorded_name = "recorded.wav"
    self.writing_name = "tone_ex"

   def set_recorded_name(self, name):
    self.recorded_name = name

   def set_writing_name(self, name):
    self.writing_name = name

   def get_data_sample_rate(self):
    sample_rate, data = wavfile.read(self.recorded_name)
    duration = len(data) / sample_rate 
    return data, sample_rate
   
   def write_sound(self, sample_rate, data):
      write(self.writing_name, sample_rate, data)

def record(record_seconds, filename):
  chunk = 1024
  FORMAT = pyaudio.paInt16
  channels = 1
  sample_rate = 44100
  p = pyaudio.PyAudio()
  stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
  frames = []
  print("Recording")
  for i in range(int(sample_rate / chunk * record_seconds)):
    data = stream.read(chunk)
    frames.append(data)
  stream.stop_stream()
  stream.close()
  p.terminate()
  wf = wave.open(filename, "wb")
  wf.setnchannels(channels)
  wf.setsampwidth(p.get_sample_size(FORMAT))
  wf.setframerate(sample_rate)
  wf.writeframes(b"".join(frames))
  wf.close()

def recording_no_stop(duration_array): # Многократная запись, которая синхронизируется с помощью текстового файла
  num = len(duration_array)
  sleep(3)
  print("RECORDING")
  i = 0
  for i in range(num):
    duration = duration_array[i]
    for l in range(10000000):
      flag = ""
      while len(flag) == 0:
        f = open("flag.txt", "r")
        flag = f.read()
        f.close()
      flag = bool(int(flag[len(flag) - 1]))      
      if flag:
        break
      sleep(0.001)
    record(duration, "recorded" + str(i))
    print("MAKING PROGRESS")
  print("DONE")
    

def get_code_array(max_length):
  code_array = []
  for i in range(4, max_length + 4, 4):
    for z in range(20):
      code = ""
      for j in range(i):
        code += str(random.randint(0,1))
      code_array.append(code)
  return code_array

def get_duration_array(code_array, change_time):
  duration_array = []
  for i in range(len(code_array)):
    code = code_array[i]
    duration = change_time * len(code) + 2.5
    duration_array.append(duration)
  return duration_array




  





class decoder:
   
   def __init__(self):
      self.sample_rate, self.freq0, self.freq1, self.change_time, self.index_range, self.draw_graph, self.max_param, self.amplitude_threshold, self.print_progress, self.check_yf = 44100, 1000, 2000, 0.5, 2, False, 8, 0.45, True, False
   
   def set_print_yf(self, flag):
     self.check_yf = flag
   
   def set_print_progress(self, flag):
     self.print_progress = flag
   
   def set_max_param(self, n):
     self.max_param = n
   
   def set_amplitude_threshold(self, param):
     self.amplitude_threshold = param
   
   def set_draw_graph(self, flag):
     self.draw_graph = flag
    
   def set_freqses(self, freq0 = 1000, freq1 = 2000):
      self.freq0, self.freq1 = freq0, freq1

   def get_freqs_indexes(self, xf):
     index_freq0, index_freq1 = int(7.5 * len(xf) / 330750 * self.freq0), int(7.5 * len(xf) / 330750 * self.freq1)
     return index_freq0, index_freq1

   def fourier_abs(self, data):
    sample_rate = self.sample_rate
    duration = len(data) / sample_rate
    xf = rfftfreq(len(data), 1 / sample_rate)
    yf = rfft(data)
    yf = yf.flatten()
    return np.array([xf, np.abs(yf)])
   
   def get_index_start_end_of_longest_ones(self, lt):
     m = 0
     index = 0
     cnt = 0
     end_index = 0
     length = 0
     for i in range(len(lt)):
       a = bool(lt[i])
       if a:
         cnt += 1
       if not a:
         if cnt > m:
           m = cnt
           index = i - cnt
           end_index = index + cnt -1
           length = end_index - index + 1
         cnt = 0
     if cnt > m:
       index = i + 1 - cnt
     return index, end_index, length
   
   def fourier_for_array(self, data_array, sample_rate):
    _, ans = self.fourier_abs(data_array[0])
    ans = [ans]
    for i in range(1, len(data_array)):
      data_window = data_array[i]
      xf_for_array, yf_window = self.fourier_abs(data_window)
      ans = np.concatenate((ans, [yf_window]), axis = 0)
    return xf_for_array, ans
   
   def abs_diff(self, yf_window, xf, freq0, freq1, range_index):
    absolute_difference = abs(self.freq0_sum(yf_window) - self.freq1_sum(yf_window)) ** 2
    return absolute_difference
   
   def calc_index(self, freq, xf):
     index = int(len(xf) / (self.sample_rate / 2) * freq)
     return index
   
   def freq_sum(self, yf, xf, freq):
     index_range = self.index_range
     index_freq = self.calc_index(freq, xf)
     s = 0
     for i in range(index_freq - index_range, index_freq + index_range + 1):
       amplitude = yf[i]
       s += abs(amplitude)
     return s
   
   def set_change_time(self, change_time):
     self.change_time = change_time

   
   def fill_in(self, yf_array, xf, freq0, freq1, amplitude_threshold, length, N0):
     data_for_prediction = pd.DataFrame(index = [0])
     data_for_prediction[[f"f{i}_freq0" for i in range(64)]] = 0
     data_for_prediction[[f"f{i}_freq1" for i in range(64)]] = 0
     amplitude_array = np.zeros((2, N0))
     freq0, freq1 = self.freq0, self.freq1
     bool_array = np.zeros(len(yf_array), dtype = bool)
     normalized_yf = self.normalize_data_array(yf_array, xf, freq0, freq1)
     for i in range(len(bool_array)):
       window = normalized_yf[i]
       amplitude_freq0, amplitude_freq1 = self.freq_sum(window, xf, freq0), self.freq_sum(window, xf, freq1)
       amplitude_array[0,i], amplitude_array[1,i] = amplitude_freq0, amplitude_freq1
     if N0 > length:
       for i in range(length):
         data_for_prediction[f"f{i}_freq0"][0] = amplitude_array[0,i]
         data_for_prediction[f"f{i}_freq1"][0] = amplitude_array[1,i]
     else:
       append_length = int((length- N0) / 2)
     self.amplitude_threshold = float(model.predict(data_for_prediction))
     amplitude_threshold_backup = amplitude_threshold
     amplitude_threshold = self.amplitude_threshold
     for i in range(len(amplitude_array[0])): 
       amplitude_freq0, amplitude_freq1 = amplitude_array[0,i], amplitude_array[1,i]
       if amplitude_freq0 + amplitude_freq1 > amplitude_threshold:
        bool_array[i] = True
     return bool_array
   
   def normalize_data_array(self, yf_array, xf, freq0, freq1):
    normalized_array = yf_array
    medium_amplitude_freq0, medium_amplitude_freq1 = self.calc_medium(yf_array, xf, freq0), self.calc_medium(yf_array, xf, freq1)
    for i in range(len(yf_array)):
      window = yf_array[i]
      normalized_array[i] = self.normalize(window, xf, medium_amplitude_freq0, medium_amplitude_freq1)
    return normalized_array
   
   def decoder_1(self, data):
     freq0, freq1 = self.freq0, self.freq1
     sample_rate, change_time, range_index = self.sample_rate, self.change_time, self.index_range #Включаем все параметры
     duration = len(data) / sample_rate      #whole time
     N0 = int(duration / change_time)        #number of windows
     data_array = self.secure_split(data, N0)         #array of arrays with sound data
     points_per_window = int(change_time * sample_rate) #Количество отсчетов за такт
     xf_for_array, yf_array = self.fourier_for_array(data_array, sample_rate)  
     yf_for_array_bool = self.fill_in(yf_array, xf_for_array, freq0, freq1, self.amplitude_threshold, 64, N0) #В этом массиве отображается длина сигнала
     rough_beginning_index, rough_end_index, N = self.get_index_start_end_of_longest_ones(yf_for_array_bool)#index where signal  starts (rough estimation)
     actual_sound = 0
     definite_sound = 0
     if N % 4 == 0: #По идее, такое должно происходить не очень часто. Смещаем сигнал пока не определится как N + 1
       if self.print_progress:
         print("Смещаем сигнал")
       while N % 4 == 0:  
         data = np.concatenate((data[int(points_per_window / 2):], np.zeros(int(points_per_window / 2))), axis = None)#Само смещение
         N0 = int(duration / change_time)       
         data_array = self.secure_split(data, N0)   
         points_per_window = int(change_time * sample_rate)
         xf_for_array, yf_array = self.fourier_for_array(data_array, sample_rate)  
         yf_for_array_bool = self.fill_in(yf_array, xf_for_array, freq0, freq1, self.amplitude_threshold, 64, N0)
         rough_beginning_index, rough_end_index, N = self.get_index_start_end_of_longest_ones(yf_for_array_bool)
     if rough_beginning_index == 0 or rough_end_index == len(yf_for_array_bool) - 1:
       return 0
     if N % 4 == 1:
       if self.print_progress:
         print("Сигнал определился нормально")
       definite_sound_array, supposed_sound_array, p = self.sound_array_by_index(data, rough_beginning_index, points_per_window, N) #Здесь мы достаем два массива: в одном полностью содержится сигнал (включая шумы),  а во второй - должен превратиться в сигнал (находится внутри первого)
       actual_sound, F_max = self.find_sound_1(N - 1, definite_sound_array, supposed_sound_array, self.max_param) #Находим сам сигнал
       code = self.decoding(actual_sound, N - 1)#Декодируем сигнал
       if self.print_progress:
         print("Определенная длина равна", N - 1)
       if self.check_yf:
         print(yf_for_array_bool)        
       if self.draw_graph:
         _, definite_yf_array = self.fourier_for_array(self.secure_split(definite_sound_array, N), sample_rate) #Эта часть необязательна: она нужна лишь для построения функции (наглядное представление)
         definite_sound, _, _ = self.sound_array_by_index(data, rough_beginning_index, points_per_window, N)
         _, m, act = self.plotting_function(definite_yf_array, definite_sound_array, supposed_sound_array, freq0, freq1, xf_for_array, range_index, N, sample_rate)
         print(m - F_max)
       definite_sound = 0
     if N % 4 == 2:
       if self.print_progress:
         print("You done goofed up")#Этого не должно быть ни при каких обстоятельствах. Почти наверняка накосячили с параметрами
       code = "rofl"
     if N % 4 == 3:
       print("Это имеет очень маленький шанс произойти. Скорее всего, допущена ошибка")
       definite_sound_array, supposed_sound_array, p = self.sound_array_by_index_special(data, rough_beginning_index, points_per_window, N)#На эту часть забейте, практически до этого не доходит, я вообще не уверен что здесь нет ошибок в коде
       actual_sound = self.find_sound(N + 1, definite_sound_array, supposed_sound_array)
       code = self.decoding(actual_sound, N + 1)
       _, definite_yf_array = self.fourier_for_array(self.secure_split(definite_sound_array, N + 1), self.sample_rate)
     return code
   
   def sound_array_by_index_special(self, data, index_left, points_per_window, N):
     definite_sound_array = data[int(points_per_window * (index_left - 1)): int(points_per_window * (N + index_left + 1))]
     supposed_sound_array = data[int(points_per_window * (index_left - 1)): int(points_per_window * (N + index_left - 1))]
     return definite_sound_array, supposed_sound_array, points_per_window
   
   
     
   
   def decoding(self, sound, N):
     code = ""
     xf, yf_array = self.fourier_for_array(self.secure_split(sound, N), self.sample_rate)
     for i in range(len(yf_array)):
       window = yf_array[i]
       amp1, amp2 = self.freq_sum(window, xf, self.freq0), self.freq_sum(window, xf, self.freq1)
       if amp1 > amp2:
         code += "0"
       if amp1 < amp2:
         code += "1"
     return code


   def find_max(self, x, sound_data, N, l, definite_sound_data):
    next_x = x
    while len(next_x) > 4:
      x_array = self.secure_split(next_x, 3)
      next_x = np.array([x_array[0], x_array[1]])
      x1, x2, x3, x4 = x_array[0][0], x_array[1][0], x_array[2][0], x_array[2][len(x_array[0]) - 1]
      F1, F2, F3, F4 = self.calc_funcions(sound_data, N, x1, x2, x3, x4, definite_sound_data, l)
      if F2 > F3:
        next_x[0], next_x[1] = x_array[0], x_array[1]
        next_x = next_x.flatten()
      else:
        next_x[0], next_x[1] = x_array[1], x_array[2]
        next_x = next_x.flatten()    
    ans_x = np.max(next_x)
    return ans_x, F2
     
   
   def find_sound(self, N, definite_sound_array, supposed_sound_array):
    target_sound_data, definite_sound_data = supposed_sound_array.flatten(), definite_sound_array.flatten()
    l, l2 = len(target_sound_data), len(definite_sound_data)
    X = np.linspace(0, l2 - l, l2 - l)
    X1, X2= self.secure_split(X,2)[0], self.secure_split(X,2)[1]
    X1, X2 = X1.astype(int), X2.astype(int)
    ans_x1, F1 = self.find_max(X1, target_sound_data, N, l, definite_sound_data)
    ans_x2, F2 = self.find_max(X2, target_sound_data, N, l, definite_sound_data)
    if F2 > F1:
      ans = ans_x2
    else:
      ans = ans_x1
    return self.sound_data_by_x(definite_sound_data, ans, l)
   
   def find_sound_1(self, N, definite_sound_array, supposed_sound_array, n):
    target_sound_data, definite_sound_data = supposed_sound_array.flatten(), definite_sound_array.flatten()
    l, l2 = len(target_sound_data), len(definite_sound_data)
    X = np.linspace(0, l2 - l, l2 - l)
    x = np.array(self.secure_split(X, n))
    ans_array = np.zeros(n)
    F_array = np.zeros(n)
    x, ans_array = x.astype(int), ans_array.astype(int)
    for i in range(n):
      Xc = x[i]
      if len(Xc) < 2:
        break
      ans_x, F = self.find_max(Xc, target_sound_data, N, l, definite_sound_data)
      ans_array[i], F_array[i] = ans_x, F
    index = np.argmax(F_array)
    max_F = np.max(F_array)
    ans = ans_array[index]
    return self.sound_data_by_x(definite_sound_data, ans, l), F

   def sound_data_by_x(self, definite_sound_data, x, l):
     ans_sound_data = definite_sound_data[int(x):int(x + l - 1)]
     return ans_sound_data
   
   def F(self, sound_array, N):
     data_array = self.secure_split(sound_array, N)
     xf, yf_array = self.fourier_for_array(data_array, self.sample_rate)
     func = self.calc_function(yf_array, xf)
     return func


       

   def calc_funcions(self, sound_data, N, x1, x2, x3, x4, definite_sound_data, l):
     sound_data1, sound_data2, sound_data3, sound_data4 = self.sound_data_by_x(definite_sound_data, x1, l), self.sound_data_by_x(definite_sound_data, x2, l), self.sound_data_by_x(definite_sound_data, x3, l), self.sound_data_by_x(definite_sound_data, x4, l)
     F1, F2, F3, F4 = self.F(sound_data1, N), self.F(sound_data2, N), self.F(sound_data3, N), self.F(sound_data4, N)
     return F1, F2, F3, F4

   def sound_array_by_index(self, data, index_left, points_per_window, N):
    definite_sound_array = data[int(points_per_window * (index_left)): int(points_per_window * (N + index_left))]
    supposed_sound_array = data[int(points_per_window * (index_left)): int(points_per_window * (N - 1 + index_left))]
    return definite_sound_array, supposed_sound_array, points_per_window
     

   def secure_split(self, d_array, N):
     if len(d_array) % N !=0:
       d_array = np.concatenate((d_array, np.zeros(N - len(d_array) % N)), axis = None)
     d_array = np.split(d_array, N)
     l = np.shape(d_array)
     if len(l) == 3:
        d_array = np.reshape(d_array, (l[0], l[1]))
     return np.array(d_array)
   

  

   def normalize(self, array, xf, medium_amplitude_freq0, medium_amplitude_freq1):
    freq0_amplitude, freq1_amplitude = 0, 0
    index_freq0, index_freq1 = self.calc_index(self.freq0, xf), self.calc_index(self.freq1, xf)
    freq0_amplitude, freq1_amplitude = self.freq_sum(array, xf, self.freq0), self.freq_sum(array, xf, self.freq1)
    array[index_freq0 - self.index_range: index_freq0 + self.index_range + 1] = 0
    array[index_freq1 - self.index_range: index_freq1 + self.index_range + 1] = 0
    array[index_freq0], array[index_freq1]  = freq0_amplitude / medium_amplitude_freq0, freq1_amplitude / medium_amplitude_freq1
    return array

   def calc_medium(self, yf_array, xf, freq):
    medium_amplitude = 0
    sum = 0
    N = len(yf_array)
    for i in range(N):
      window = yf_array[i]
      amplitude = self.freq_sum(window, xf, freq)
      sum += abs(amplitude)
    medium_amplitude = sum / N
    return medium_amplitude
   
   def make_step(self, definite_sound_array):
     step = len(definite_sound_array) * 0.0001
     return int(step)
   
   def apply_step(self, definite_sound_array, supposed_sound_array, N, step, number_of_steps, sample_rate):
     changed_sound_array = definite_sound_array[int(step * number_of_steps + step):int(step * number_of_steps + len(supposed_sound_array) + step)]
     changed_sound_data_array = self.secure_split(changed_sound_array, N)
     xf, changed_yf_array = self.fourier_for_array(changed_sound_data_array, sample_rate)
     return xf, changed_yf_array, changed_sound_array
   
   def plotting_function(self, definite_yf_array, definite_sound_array, supposed_sound_array, freq0, freq1, xf, range_index, N, sample_rate):
     number_of_steps = 0
     y_array = np.array([], dtype = float)
     step = self.make_step(definite_sound_array)
     x_array= np.array([], dtype = float)
     while int(step * number_of_steps) + len(supposed_sound_array) + step < len(definite_sound_array) - 1 - step:
        xf, yf_array, changed_sound_array = self.apply_step(definite_sound_array, supposed_sound_array, N, step, number_of_steps, sample_rate)
        func = self.calc_function(yf_array, xf)
        y_array = np.concatenate((y_array, [func]), axis = None)
        step = self.make_step(definite_sound_array)
        supposed_sound_array = changed_sound_array
        x_array = np.concatenate((x_array, [number_of_steps]), axis = None)
        number_of_steps += 1
     plt.plot(y_array)
     m = np.argmax(y_array)
     return x_array, np.max(y_array), step*(x_array[m])
   
     
   
   def abs_diff(self, yf_window, xf, freq0, freq1):
    absolute_difference = (self.freq_sum(yf_window, xf, freq0) - self.freq_sum(yf_window, xf, freq1))** 2
    return absolute_difference
   
   def calc_function(self, yf_array, xf):#Все функции снизу нужны для собирания данных
    freq0, freq1 = self.freq0, self.freq1
    func = 0
    for i in range(len(yf_array)):
      yf_window = yf_array[i]
      func += self.abs_diff(yf_window, xf, freq0, freq1)
    return func
   
   def testing_decoder(self, data, true_N):#Этот код очень (прям совсем) не хорошо оптимизирован, но это не важно.
     freq0, freq1 = self.freq0, self.freq1
     sample_rate, change_time, range_index = self.sample_rate, self.change_time, self.index_range #Включаем все параметры
     duration = len(data) / sample_rate      #whole time
     N0 = int(duration / change_time)        #number of windows
     data_array = self.secure_split(data, N0)         #array of arrays with sound data
     points_per_window = int(change_time * sample_rate) #Количество отсчетов за такт
     amplitude_threshold = 1
     xf_for_array, yf_array = self.fourier_for_array(data_array, sample_rate)  
     yf_for_array_bool = self.fill_in(yf_array, xf_for_array, freq0, freq1, amplitude_threshold, 64, N0) #В этом массиве отображается длина сигнала
     _, _, N = self.get_index_start_end_of_longest_ones(yf_for_array_bool)
     rare_flag = 0
     while True:
       if N < true_N - 1:
         amplitude_threshold -= 0.01
       if N > true_N + 1:
         amplitude_threshold += 0.01
       if N % 4 == 0:
         j = 0
         data_backup = data
         while N % 4 == 0:
           if j > 2:
             break
           data = np.concatenate((data[int(points_per_window / 2):], np.zeros(int(points_per_window / 2))), axis = None)#Само смещение
           N0 = int(duration / change_time)       
           data_array = self.secure_split(data, N0)   
           xf_for_array, yf_array = self.fourier_for_array(data_array, sample_rate)  
           yf_for_array_bool = self.fill_in(yf_array, xf_for_array, freq0, freq1, amplitude_threshold, 64, N0)
           _, _, N = self.get_index_start_end_of_longest_ones(yf_for_array_bool)
           if yf_for_array_bool[0] == True:
             break
           j += 1
         data = data_backup
       if N % 4 == 1:
         break
       if N % 4 == 3:
         if rare_flag > 50:
           break
         rare_flag += 1
         amplitude_threshold -= 0.01
       if N % 4 == 0:
         break
       yf_for_array_bool = self.fill_in(yf_array, xf_for_array, freq0, freq1, amplitude_threshold, 64, N0)
       _, _, N = self.get_index_start_end_of_longest_ones(yf_for_array_bool)
       if amplitude_threshold < 0:
         return 0.45
     return amplitude_threshold

def get_max_param(data, true_code, amplitude_threshold):
  a = decoder()
  a.set_amplitude_threshold(amplitude_threshold)
  a.set_freqses(1000, 1500)
  a.set_change_time(0.1)
  a.set_draw_graph(False)
  a.set_print_progress(False)
  code = ""
  i = 1
  while code != true_code:
    a.set_max_param(i)
    code = a.decoder_1(data)
    i += 1
    if i >= 100:
      break
    
  return i


  
def get_max_param_threshold(data_array,code_array):
  a = decoder()
  amplitude_threshold_array = []
  max_param_array = []
  a.set_print_progress(False)
  for i in range(0, len(code_array)):
    data = data_array[i]
    code = code_array[i]
    N = len(code)
    amplitude_threshold = a.testing_decoder(data, N)
    max_param = get_max_param(data, code, amplitude_threshold)
    amplitude_threshold_array.append(amplitude_threshold)
    max_param_array.append(max_param)
    print("MAKING PROGRESS")
  return max_param_array, amplitude_threshold_array

def correction_array(code_array):
  b = wav_manager()
  a = decoder()
  a.set_print_yf(False)
  a.set_freqses(1000, 1500)
  a.set_change_time(0.1)
  a.set_max_param(8)
  a.set_draw_graph(False)
  a.set_print_progress(False)
  true_data_array = []
  true_code_array = []
  for i in range(len(code_array)):
    b.set_recorded_name("recorded" + str(i))
    data, _ = b.get_data_sample_rate()
    code = code_array[i]
    l = len(code)
    print(code)
    amp = a.testing_decoder(data, l)
    a.set_amplitude_threshold(amp)
    max_param = get_max_param(data, code, amp)
    a.set_max_param(max_param)
    codi = a.decoder_1(data)
    if codi == code:
      true_data_array.append(data)
      true_code_array.append(code)
      print(i, "-th RECORDED FILE DECODED CORRECTLY")
    else:
      print("ERROR IN DECODING FOR", i, "-th")
    
  return true_data_array, true_code_array

    






    
    
    
    


       
       




          

In [16]:
code_array = get_code_array(64)#Здесь рандомным (почти) образом идет генерация двоичного кода, который кодируется в сигнал
duration_array = get_duration_array(code_array, 0.1)
code_array = ['1100',
 '0010',
 '1011',
 '1000',
 '1011',
 '1111',
 '0100',
 '1000',
 '1111',
 '1000',
 '1111',
 '1010',
 '0111',
 '0110',
 '1101',
 '0010',
 '1011',
 '0110',
 '1100',
 '1111',
 '11101010',
 '00100010',
 '01111000',
 '00110011',
 '00001111',
 '01000110',
 '11000110',
 '11110100',
 '10010001',
 '01100010',
 '00000111',
 '10101100',
 '01101110',
 '10110001',
 '01101101',
 '10000101',
 '00110010',
 '01001111',
 '11110110',
 '00011011',
 '011000100010',
 '110101100101',
 '101000010001',
 '101010011100',
 '111010010000',
 '010011000100',
 '100000111111',
 '101011001100',
 '101101101111',
 '101011010100',
 '101100000011',
 '011111110101',
 '000001101011',
 '011101011110',
 '010101010001',
 '111110111001',
 '100001000011',
 '010110111100',
 '010000101001',
 '110001000001',
 '0001111011001001',
 '1000000110110011',
 '0110001010011011',
 '0101011111100011',
 '1010011011011100',
 '0011110011010000',
 '1010000100000110',
 '0011000000110000',
 '1001000111010010',
 '1100111101011011',
 '1111000100101011',
 '1110010001101011',
 '0111010011011000',
 '0000011001000111',
 '1110111000111000',
 '1110010010000010',
 '0101001111000011',
 '1000010110001010',
 '0000000010010100',
 '1111011011010000',
 '10001010100011101101',
 '11111011001000101110',
 '01111011101000110111',
 '11001100010011000100',
 '10110100001110000001',
 '00000001010011100100',
 '10101010010000000000',
 '10101100000100001101',
 '01101110001111011010',
 '01010011110000110111',
 '11100100000000100111',
 '10010001101110100101',
 '10001101101001100101',
 '10101111100011010001',
 '01001100101000010111',
 '10101110001110001000',
 '01101011100011110111',
 '00001110011111011001',
 '10111100110011001000',
 '00111010000110111011',
 '101011100100111011101010',
 '001000111001011110100011',
 '101000000000100111101100',
 '011010000111000110111000',
 '010011101011110110011110',
 '000000010010101001011101',
 '111010101000001111111011',
 '100000001010110110111110',
 '001001011010111010110000',
 '011110111011111011101000',
 '101111101101110010001000',
 '100111000100011111011101',
 '100001101000011100101010',
 '010110101010111001100111',
 '010001001101111011011000',
 '100110010010101010111011',
 '101100001001001101001000',
 '100010100111000111011000',
 '010101001101000110101100',
 '000100000010011001010101',
 '0111100010000000111101001010',
 '1111010010010111110111101011',
 '1100110001010111100010001001',
 '0011001101100110111010010010',
 '0000000011010100000000101011',
 '1110100001000101111000000001',
 '0010101010010010100011101010',
 '1000010110000000101100011100',
 '1001100011101110001100000001',
 '1011001101101010000011100011',
 '1000101111101011110001110000',
 '0101100100101101011111111100',
 '0111010011000000010010111011',
 '1100111101000110011011010001',
 '0010111111011100011100001000',
 '1101100011000111101111010000',
 '0100010100110010010011100000',
 '0111110100101000001001111011',
 '0011011111110111000011111001',
 '0111111000010011001010011010',
 '10011011101011011000101001100001',
 '11010011110110011100010101110011',
 '01111110111000000001101011100000',
 '00100101100101000100000000010100',
 '01111111010010010000010110111100',
 '10000100000001001100010110000100',
 '10110001000011111110000000100011',
 '10110000010100110111001010001110',
 '01000101100001100011101000100100',
 '11101001110100111011001010000011',
 '00011111101010001111101110100010',
 '10100000111111110010001110111111',
 '10100111111101111100111110111011',
 '00111100101111101011010011000101',
 '10011110010111110100001001001000',
 '00000011111110011001010110011010',
 '00011001000011110111000100010011',
 '10000000010110011000000001110001',
 '01111101110010100000101110100100',
 '11111000010011010011111100110110',
 '101000010011000110011011001000101011',
 '011001110101100010100111010001100110',
 '001011111010011111001100011011010000',
 '100100110100111111101001111101010100',
 '100001101010000110100011010000011000',
 '110001100111001000000001011000100010',
 '001010010100000010001010000100000111',
 '100101011110100110011001010110000010',
 '000011111100111110100001100111011110',
 '110010000110110000010000000110010100',
 '001101010111001101100110111111110010',
 '111110001100010100111101101111000100',
 '011010110100101001000010110110001111',
 '101101110100101000010110000100000111',
 '010010111011100001101011110001011101',
 '111011111110001010110011010011010010',
 '011110000011100001101010000111000100',
 '001101110011111101011011000101010011',
 '110100100110011111000110001010001110',
 '111101011011100011000100001000001011',
 '1101111110010110110100010110011100111010',
 '1010100011110010101110100110100010010111',
 '1111100101101111100100101110110010100000',
 '1100011000010100101111101000010001010001',
 '0000100000110011011101010111010100111100',
 '1111010000000101111100110110011011101101',
 '1000001001011011110100011101111111101110',
 '1001110101011010110110010100110101001011',
 '0101111010100011010100101100101101011111',
 '1101100000000100110001100001100100001101',
 '0111110110111010011000110001101100010000',
 '1010000110100111001010111100000000000111',
 '1110010101000100100110000010011101001011',
 '1010001110000010011110000100101000001101',
 '0100101001011110100101000011000001100101',
 '0100111001110110101101110010011110001010',
 '1101110101001001101000111100011100101101',
 '0001010100011100111000001001110011111000',
 '1011100110010111110111110111000010100101',
 '1100001101011010000100110001111010111010',
 '10000111110010000011010110101011111011001010',
 '01011110000000111011100011111011101101000010',
 '00000010010011000011110111001100010011001110',
 '01111001101100000000011000110001001000101000',
 '00000000100001110110101000001001010111011101',
 '11010001101001111000001110111100100101011110',
 '10001110011001101001011101111001100111110010',
 '01111010111011011110011000100001010001011110',
 '11111111111000001010011110110001111101001101',
 '00000011001111000101011001000001100010000010',
 '00111011111010100000101011111010100000010101',
 '00001110001000110011100100010110111011110111',
 '01100001001010010010011111000010011000111100',
 '00011011111111110000100001110100110111100100',
 '01100000100101110001100110110001110100001010',
 '00001101100000110010101111100101101011101011',
 '00110010100110011110011111111001001110101001',
 '01000011111000011100001111110011010010001010',
 '00110100101001000110111100011010000001100010',
 '10111010010001101111011000100000010000111001',
 '101101100110100011110111100001000010101110111101',
 '001001111001001110110010010100001100001000010000',
 '100101100110010110110100101010100010101110100001',
 '110111000101110010111110010111100101100101100010',
 '010100101001000111110000100000110001001001011100',
 '101011001011010011110010110100010011110010001001',
 '011111010011110111110110110010001100100101101101',
 '001100100111001110111111010101000110110111010100',
 '101000000001100000001111001001101011100101110111',
 '011001111111001001100000000010011010100101011001',
 '001110011001101110111011100011010001000100000010',
 '001100000111000010100101000000010011111000110010',
 '011001111110111001101001000111110000111101100000',
 '100100010010001110100111100101011111001000010101',
 '001100000000111000001000000101100011110010011111',
 '010101100101110000011000011110001011110011000010',
 '100001001110110111010000111100110010111010000100',
 '000110111100111011001110010010011100011000001010',
 '011010010001010111011010101111111001010000010001',
 '101000100100011011101001010010011010110010010100',
 '0011001010001001100011000101010100000011111100110101',
 '1101110101100001110001001000100011001100011001110100',
 '1100000101000101100100000001110011011011110010101011',
 '1010001000000010101111011100100100001101101111111001',
 '0001110000101010000001101001100011100110101101000011',
 '1111000001110111111101100101111001011010010011111100',
 '1110100001111010110100110100101101010011101111110100',
 '1101101100101001000010101101110111001101011110000000',
 '0000001011011110100001100111101100110100111101001011',
 '1000000111101100100010110010101010001000010100101111',
 '1110110001101101001110111000000100001010101001101011',
 '1110001111000001101010110010000101101000111110111010',
 '1110000001001000101100000011010010111011010011001110',
 '0001000110001111010010110111101010100111001100010110',
 '0101001010001100000011101010110000001001010010001111',
 '1100100110011010100011011000001110010011011000110110',
 '1100000011100011001000100111100010011101011100111000',
 '0110001001001001010100101101010011111000011000110010',
 '0100101010101010100000001110111001100100110110000101',
 '0100100110110001010110110000101001000010111000100111',
 '11010001000110101101010110000101011111100001110010111111',
 '01000111111100110100010001110101101101110101000110110000',
 '00000111000111011001111101111101101011011000011100010011',
 '00101110011000111011001101110011010000001100001001001001',
 '01011000100000111000100011010001100011010110101111110011',
 '11000111001000111111101001110001000000010101101110111101',
 '01001001111010111110000101010011011000110001100000001011',
 '01011100000110001001010001010000101110011001101010011110',
 '01001011111001010011000101011001000101101000010010000101',
 '10111100001001101010111010100011111110111010000111100110',
 '00000001101111100100111110010011010000001011100001001011',
 '01011101111101101100110011010001100001000000100000001010',
 '10110011110010011011001010101111101101001100101010101101',
 '00001111000010101100101111001001010001001010010001101101',
 '00000101110011010001001111001011101001101110001001011101',
 '10100011111110001001100110100110111000011000100101110100',
 '00111111110101011110001110111110100100110000001110010010',
 '01011101100011000001000100100111111100100010100010101000',
 '01110010001000111000101010111001101100000011010111000110',
 '00111011000101101100101001000010110110110011000100110001',
 '111011010100000111011100111011000111001101111110101111010111',
 '011010110011010000010100001011001111101011001111011101111101',
 '101011111100101011111101010100011001110111011110011011000100',
 '111010101100010100001101010010001000101010011000010111001110',
 '100110001110001101111010010010100010101100000111100110011101',
 '000100111011001010010011111110010111011110111011111000101111',
 '111101001001111010100111111100110111001010110111110001010001',
 '100000010010101011011001001001010111100010010101001011101000',
 '100011011001011100010001100011001110110100001101000011011010',
 '010100000111110000110111010011001101110010111110110101110010',
 '110110010011101010101100110000110000001111101001000001110000',
 '010001000111011001001011010000000111110100011110011011101010',
 '101000111011001000000001010000100011010100100101101000011110',
 '001001111101100111101101011100010001011011001111000010101011',
 '111100101101000010110100101111101100000111100011111000101000',
 '000111111110010111010101011010010011011110111001110000010101',
 '000111101000010010011001011000101111101111001100111011010110',
 '101010101000010000111101110001110001100100001110000111000111',
 '110110001010011011101010110000100010101010010100001010111100',
 '101010111010011011000110101100011111010101101111000000001010',
 '0111100000011011000101101000111011001000010010110010001010101010',
 '0100001011010110000010110100000000100111100101010010011001101010',
 '1110010011001000000110111011111010010001110000011000011011011000',
 '1100001100000010011100011010111111011111010111100101111100000000',
 '0100010101111110001001111100100010000010010001100111010111011010',
 '0011000010010101111000111011100001101111001010011000001010001110',
 '1110010100011100010100101110000101111100001000101010011111011010',
 '1110101110111101111100101011011111010011011001011001010001111110',
 '0010011100001000000011100001011100111110110000101100100010000010',
 '0100000100011011001100111100000110010011001100100010100100100101',
 '1000000111011010100110011010110101101101101001010111100001101010',
 '1011011100101100100111001101100110010000110101101101111111011011',
 '1110100000011101001001101111101111000101010111100100100010000100',
 '0110111111010101101111010010101011100000001000111011110010100110',
 '1001001101001010111010010110111101101101001001010111011100010101',
 '1101000011110001100001000001000100001111110101100010010000101110',
 '0001100100110000011110110010001011101101010101010111111000000111',
 '1010110011010111101001100111000001001000110000101010110101011111',
 '1000001110010110000001010111011111101100001111001110110110000111',
 '0010111011000001101111011011011001101010001101110011101111100011']

In [251]:
a = decoder()#Запустите эту ячейку, чтобы декодировать сигнал. Ниже идет собирание, обработка данных и обучение модели
b = wav_manager()
b.set_recorded_name("recorded.wav")
data, _ = b.get_data_sample_rate()
a.set_freqses(1000, 1500)#Частоты. Планируется поэксперементировать с ними потом.
a.set_change_time(0.1)#Поделите единицу на это - будет скорость сигнала (сейчас это 10 символов\секунда). Пытался попробовать меньше - распознается хуже. Если выбрать меньшую скорость, распозноваться должно с большего расстояния и/или с меньшей громкости.
a.set_max_param(8)#Насколько точно определяется максимум. В идеальном мире он будет определяться абсолютно точно независимо от этого значения.
a.set_draw_graph(False)#Рисовать ли график максимизируемой функции.
a.set_amplitude_threshold(1)#Вообще ни на что не влияет, т.к пороговое значение все равно определяется (предсказывается) внутри декодировщика.
codi = a.decoder_1(data)#Сам декодированный код.
codi

Сигнал определился нормально
Определенная длина равна 32


'10101011101010111010101110101011'

In [29]:
td, tc = correction_array(code_array)#Обработка (и выбраковка) записанных данных. В идеале, чем больше это времени займет, чем лучше. Плохо, что на 300 сигналов понадобилось 2 часа...
p, amplit = get_max_param_threshold(td, tc)
len(td)

1100
0 -th RECORDED FILE DECODED CORRECTLY
0010
1 -th RECORDED FILE DECODED CORRECTLY
1011
2 -th RECORDED FILE DECODED CORRECTLY
1000
3 -th RECORDED FILE DECODED CORRECTLY
1011
4 -th RECORDED FILE DECODED CORRECTLY
1111
5 -th RECORDED FILE DECODED CORRECTLY
0100
6 -th RECORDED FILE DECODED CORRECTLY
1000
7 -th RECORDED FILE DECODED CORRECTLY
1111
8 -th RECORDED FILE DECODED CORRECTLY
1000
9 -th RECORDED FILE DECODED CORRECTLY
1111
10 -th RECORDED FILE DECODED CORRECTLY
1010
11 -th RECORDED FILE DECODED CORRECTLY
0111
12 -th RECORDED FILE DECODED CORRECTLY
0110
13 -th RECORDED FILE DECODED CORRECTLY
1101
14 -th RECORDED FILE DECODED CORRECTLY
0010
15 -th RECORDED FILE DECODED CORRECTLY
1011
16 -th RECORDED FILE DECODED CORRECTLY
0110
17 -th RECORDED FILE DECODED CORRECTLY
1100
18 -th RECORDED FILE DECODED CORRECTLY
1111
19 -th RECORDED FILE DECODED CORRECTLY
11101010
20 -th RECORDED FILE DECODED CORRECTLY
00100010
21 -th RECORDED FILE DECODED CORRECTLY
01111000
22 -th RECORDED FILE DECO

283

In [30]:
p, amplit = get_max_param_threshold(td, tc)#еще данные

MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING PROGRESS
MAKING P

In [63]:

True_Training_Data = np.zeros(shape = (283, 64, 3065))#Преобразование фурье для собранных данных
for i in range(len(td)):
    data = np.array(td[i])
    padded_data = np.concatenate((data, np.zeros(len(td[282]) - len(data))))
    Training_XF_Data, fourier_padded_data = a.fourier_for_array(a.secure_split(padded_data, 64), 44100)
    True_Training_Data[i] = fourier_padded_data

In [72]:
Train_freq0_array = np.zeros((283, 64)) #"Заполняем" некоторые значение (паддинг), т.к сигналы разной длины
Train_freq1_array = np.zeros((283, 64))
for i in range(len(True_Training_Data)):
    yf_array = True_Training_Data[i]
    yf_array = a.normalize_data_array(yf_array, Training_XF_Data, 1000, 1500)
    for j in range(64):
        yf = yf_array[j]
        amp_freq0, amp_freq1 = a.freq_sum(yf, Training_XF_Data, 1000), a.freq_sum(yf, Training_XF_Data, 1500)
        Train_freq0_array[i, j], Train_freq1_array[i, j] = amp_freq0, amp_freq1



array([ 0.12292063,  0.15444899,  0.22944664,  0.40969599,  0.16768233,
        0.10594161,  0.10750614,  0.14035761,  0.06510982,  0.13959211,
        0.09998068,  0.37656988, 26.71314506, 23.59287071, 10.62904892,
        0.31438121,  0.16935199,  0.14750547,  0.10876791,  0.12411104,
        0.08156525,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

In [101]:
TrainingDataset = pd.DataFrame(index = np.linspace(0, 282, 283))
TrainingDataset[[f"f{i}_freq0" for i in range(64)]] = 0
TrainingDataset[[f"f{i}_freq1" for i in range(64)]] = 0
TrainingDataset["max_param"] = 0
TrainingDataset["amp_threshold"] = 0
TrainingDataset["length"] = 0
for i in range(64):
    for j in range(283):
        TrainingDataset[f"f{i}_freq0"][j] = Train_freq0_array[j,i]
        TrainingDataset[f"f{i}_freq1"][j] = Train_freq1_array[j,i]
        TrainingDataset["max_param"][j] = p[j]
        TrainingDataset["amp_threshold"][j] = amplit[j]
        TrainingDataset["length"][j] = len(tc[j])
TrainingDataset#Создание конечного датасета

/tmp/ipykernel_5807/346232136.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  TrainingDataset[[f"f{i}_freq1" for i in range(64)]] = 0
/tmp/ipykernel_5807/346232136.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  TrainingDataset[[f"f{i}_freq1" for i in range(64)]] = 0
/tmp/ipykernel_5807/346232136.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

,f0_freq0,f1_freq0,f2_freq0,f3_freq0,f4_freq0,f5_freq0,f6_freq0,f7_freq0,f8_freq0,f9_freq0,...,f57_freq1,f58_freq1,f59_freq1,f60_freq1,f61_freq1,f62_freq1,f63_freq1,max_param,amp_threshold,length
0.0,0.122921,0.154449,0.229447,0.409696,0.167682,0.105942,0.107506,0.140358,0.065110,0.139592,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.99,4
1.0,0.042877,0.045239,0.061720,0.063699,0.052829,0.095235,0.051877,0.066920,0.222249,0.052747,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.54,4
2.0,0.119038,0.119468,0.125734,0.149360,0.096273,0.237518,0.151898,0.142295,0.185614,0.133377,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.76,4
3.0,0.120825,0.159579,0.097711,0.101082,0.151825,0.186910,0.087471,0.367049,0.190753,0.132302,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.82,4
4.0,0.126829,0.184287,0.141417,0.193877,0.103230,0.048362,0.163119,0.209220,0.182319,0.122975,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.63,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278.0,0.003451,0.004447,0.008657,0.006718,0.008872,0.003921,0.005415,0.008062,0.006881,0.006145,...,1.622959,0.073616,0.010136,0.003677,0.005632,0.003502,0.002715,2,0.99,64
279.0,0.006634,0.004733,0.008560,0.003606,0.005768,0.007504,0.009368,0.008457,0.009200,0.008119,...,1.762585,0.093952,0.012084,0.005941,0.003292,0.005011,0.004733,2,0.99,64
280.0,0.007051,0.008490,0.010663,0.009003,0.008198,0.009136,0.007512,0.007514,0.007322,0.009069,...,0.181647,0.036029,0.003480,0.003514,0.004506,0.004664,0.004358,2,0.99,64
281.0,0.007917,0.006686,0.006220,0.005961,0.006623,0.010409,0.005074,0.009230,0.010030,0.006652,...,2.291145,1.125921,0.036541,0.007514,0.004069,0.005200,0.002881,2,0.99,64


In [201]:
X = TrainingDataset.drop(columns = ["max_param", "amp_threshold", "length"])
y = TrainingDataset["amp_threshold"]
y

0.0      0.99
1.0      0.54
2.0      0.76
3.0      0.82
4.0      0.63
         ... 
278.0    0.99
279.0    0.99
280.0    0.99
281.0    0.99
282.0    0.99
Name: amp_threshold, Length: 283, dtype: float64

In [252]:
TrainingDataset.to_csv("Dataset.csv", index = False)

In [202]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)#Т.к данных собрано (пока что) не очень много, выбран достаточно большой коэффициент

print(f'Train dataset size: {X_train.shape}, {y_train.shape}')
print(f'Test dataset size: {X_test.shape}, {y_test.shape}')

Train dataset size: (56, 128), (56,)
Test dataset size: (227, 128), (227,)


In [203]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [204]:
y_pred = model.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))

Mean Absolute Error: 0.4041760764896646
Mean Squared Error: 0.30042956288209366
